# Extract
> Datasource to DataLake(HDFS)
>
> Coupang website에서 web crawling

In [2]:
# !pip install hdfs

In [22]:
from hdfs import InsecureClient # hadoop 서버와 연결하는  client객체 모듈(hdfs 접근)
import requests # web 사용
import json # json 기능 사용
import datetime as dt

## DL => DW
- 필요 data Lake에서 HDFS로 이동 저장 단계
- 쿠팡 웹사이트 web 페이지 크롤링



In [23]:
## hdfs client 연결객체 생성
client=InsecureClient('http://localhost:9870',user='lab04')

Logger 사용

In [3]:
import logging

In [4]:
# 현재 날짜로부터 befor_day 만큼 이전의 날짜를 생성해주는 함수
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [5]:
cal_std_day(365)

'2023-01-26'

In [6]:
co_logger = logging.getLogger('coupang')
f_handler = logging.FileHandler('./log/coupang_crawl/'+cal_std_day(0)+'.log')
co_logger.addHandler(f_handler)

In [7]:
# !pip install --upgrade typing-extensions

In [8]:
# !pip install selenium
# !pip install requests
# !pip install bs4

In [9]:
import pandas as pd
import numpy as np
import os

# 패키지 임포트
import selenium
import pandas as pd
import numpy as np
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 함수(필수)
import time
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.service import Service

import requests

In [10]:
def extract_products():
    url = 'https://www.coupang.com'
    headers = {"User-Agent" : "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.3) Gecko/2008092416 Firefox/3.0.3"}
    
    res = requests.get(url, headers=headers)
    html = res.text
    bs_obj = BeautifulSoup(html, 'html.parser')
    # print(bs_obj.prettify())
    
    categories = bs_obj.find('ul', {'class': 'menu shopping-menu-list'})\
                    .select('#gnbAnalytics > ul.menu.shopping-menu-list > li > a')
    first_categories = []
    for category in categories:
        first_categories.append(category.text)
    
    # 두번째 depth categories
    second_category = []
    first = bs_obj.select('#gnbAnalytics > ul.menu.shopping-menu-list > li')

    for f in first:
        category = f.find('a').text
        second = f.findAll('li', {'class': 'second-depth-list'})
        for s in second:
            second_cat = s.find('a').text
            if second_cat == '더보기':
                continue
            second_category.append([category, second_cat, 'http://www.coupang.com' + s.find('a')['href']])


    one = []
    two = []
    three = []
    for elem in second_category:
        one.append(elem[0])
        two.append(elem[1])
        three.append(elem[2])

    cat_urls = pd.DataFrame({'카테고리1': one, '카테고리2': two, 'url': three})
    # cat_urls.to_csv('./category_urls.csv', index=False, encoding="utf-8-sig")
    
    products_list = pd.DataFrame({'카테고리1': [], '카테고리2': [], '카테고리 url': [], '제품명': [], '제품 url': [], '리뷰 개수': [], '상품 별점': []})
    for category in second_category:
        print(category)
        try:
            url = category[2] + '?page=1'
            headers = {"User-Agent" : "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.3) Gecko/2008092416 Firefox/3.0.3"}
            res = requests.get(url, headers=headers)
            html = res.text

            bs_url = BeautifulSoup(html, 'html.parser')

            page_cnt = bs_url.find('div', {'id': 'product-list-paging'})['data-total']
            page_cnt = int(page_cnt)


            for i in range(1, page_cnt + 1):  # page_cnt + 1
                page_url = url[:-1] + str(i)
                headers = {"User-Agent" : "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.3) Gecko/2008092416 Firefox/3.0.3"}
                res = requests.get(page_url, headers=headers)
                html = res.text

                bs_page = bs4.BeautifulSoup(html, 'html.parser')
                # print(page_url)

                product_url = bs_page.find_all('a', {'class': 'baby-product-link'})
                product_name = bs_page.find_all('dt', {'class': 'image'})
                review_cnt = bs_page.find_all('span', {'class': 'rating-total-count'})
                product_star = bs_page.find_all('em', {'class': 'rating'})
                # print(product_url, product_name)

                # 제품명 scrapping
                result = []

                for ul, name, cnt, star in zip(product_url, product_name, review_cnt, product_star):
                    ul = 'http://www.coupang.com' + str(ul['href'])
                    name = name.find('img')['alt']
                    cnt = cnt.text[1:-1]
                    result.append(pd.DataFrame({'카테고리1':category[0], '카테고리2':category[1], '카테고리 url':url,'제품명': name, '제품 url': ul, '리뷰 개수': cnt, '상품 별점': star},
                                                index=range(0,1)))

                single_page = pd.concat(result).reset_index(drop=True)

                products_list = pd.concat([products_list, single_page]).reset_index(drop=True)
        except:
                time.sleep(2)

                url = category[2] + '?page=1'
                headers = {"User-Agent" : "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.3) Gecko/2008092416 Firefox/3.0.3"}
                res = requests.get(url, headers=headers)
                html = res.text

                bs_url = BeautifulSoup(html, 'html.parser')

                page_cnt = bs_url.find('div', {'id': 'product-list-paging'})['data-total']
                page_cnt = int(page_cnt)


                for i in range(1, page_cnt + 1):
                    page_url = url[:-1] + str(i)
                    headers = {"User-Agent" : "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.3) Gecko/2008092416 Firefox/3.0.3"}
                    res = requests.get(page_url, headers=headers)
                    html = res.text

                    bs_page = bs4.BeautifulSoup(html, 'html.parser')
                    # print(page_url)

                    product_url = bs_page.find_all('a', {'class': 'baby-product-link'})
                    product_name = bs_page.find_all('dt', {'class': 'image'})
                    review_cnt = bs_page.find_all('span', {'class': 'rating-total-count'})
                    product_star = bs_page.find_all('em', {'class': 'rating'})
                    # print(product_url, product_name)

                    # 제품명 scrapping
                    result = []

                    for ul, name, cnt, star in zip(product_url, product_name, review_cnt, product_star):
                        ul = 'http://www.coupang.com' + str(ul['href'])
                        name = name.find('img')['alt']
                        cnt = cnt.text[1:-1]
                        result.append(pd.DataFrame({'카테고리1':category[0], '카테고리2':category[1], '카테고리 url':url,'제품명': name, '제품 url': ul, '리뷰 개수': cnt, '상품 별점': star},
                                                    index=range(0,1)))

                    single_page = pd.concat(result).reset_index(drop=True)

                    products_list = pd.concat([products_list, single_page]).reset_index(drop=True)
    return products_list

# 리뷰 데이터 크롤링 코드


In [11]:
# 크롤링해 올 리뷰 개수 조정을 위한 코드
def crawl_star_cnt(url):
    # url = 'https://www.coupang.com/vp/products/7350133859?itemId=18912258874&vendorItemId=82079311639&sourceType=sdp_carousel_3_ads&clickEventId=71c3eb60-aa43-11ee-9730-43f77ae31272&templateId=2114&isAddedCart='
    driver = webdriver.Chrome()
    driver.set_window_size(500, 500)
    # time.sleep(2)
    driver.get(url)
    time.sleep(2)
    driver.set_window_size(500, 500)
    
    driver.find_element(By.XPATH, '//*[@id="btfTab"]/ul[1]/li[2]').click()  # 상품평 클릭
    time.sleep(5)
    
    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star.js_reviewArticleSearchStarSelectBtn').click()
    time.sleep(2)
    star_review = driver.find_elements(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item.js_reviewArticleStarSelectOption')
    cnt = []
    for i in range(1,6):
        star_cnt = driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(i))
        cnt.append(star_cnt)
    # 5개의 별점 분류체제의 각 별점별로 리뷰의 개수 가져오기
    star_distribution = []
    for lvl, ct in zip(star_review, cnt):
        level = lvl.get_attribute('data-rating')
        count = ct.text.replace(',', '')
        star_distribution.append([level, count])
    driver.close()
    
    # driver.close()
    
    # 3개의 분류체제로 변형하기
    star_three = []
    negative = 0
    for i in star_distribution:
        if i[0] == '5':
            star_three.append(['좋음', int(i[1])])
        if i[0] == '4':
            star_three.append(['보통', int(i[1])])
        if (i[0] == '3') | (i[0] == '2') | (i[0] == '1'):
            negative += int(i[1])
    star_three.append(['별로', negative])
    star_three
    return star_three

In [12]:
# 좋음, 보통, 별로 중 가장 리뷰의 개수가 적은 데이터를 뽑는다.
# 그렇게 해서 구한 가장 적은 리뷰의 개수가 100개 이상일 때, 좋음, 보통, 별로 3개에 대한 리뷰를 각각 100개씩만 가져온다.
# 가장 작은 리뷰의 개수가 최소 5개 이상은 되는 상품만 이용
def min_star(url):
    star_cnt = crawl_star_cnt(url)
    min = star_cnt[0][1]
    for i in star_cnt:
        if i[1] < min:
            min = i[1]
            star_level = i[0]
    if min > 42:  # 가장 적은 리뷰의 개수가 60개 이상이면 min을 60으로 정한다
        min = 42
    return [star_level, min]



In [13]:
def get_star_page(url):
    driver = webdriver.Chrome()
    driver.set_window_size(500, 500)
    # time.sleep(2)
    driver.get(url)
    time.sleep(2)
    
    min = min_star(url)
    
    driver.find_element(By.XPATH, '//*[@id="btfTab"]/ul[1]/li[2]').click()  # 상품평 클릭
    time.sleep(2)
    
    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star.js_reviewArticleSearchStarSelectBtn').click()
    

In [14]:
def review_data(reviews):
    product_reviews = []
    # try:
    for review in reviews:
        # name = review.find('span', class_='sdp-review__article__list__info__user__name js_reviewUserProfileImage')
        # member_id = review.find('span', class_='sdp-review__article__list__info__user__name js_reviewUserProfileImage')['data-member-id']
        # star = review.find('div', class_='sdp-review__article__list__info__product-info__star-orange js_reviewArticleRatingValue')['data-rating']
        # title = review.find('div', class_='sdp-review__article__list__headline')
        # contents = review.find('div', class_='sdp-review__article__list__review__content js_reviewArticleContent')
        # help = review.find('span', class_='js_reviewArticleHelpfulCount')
        # date = review.find('div', class_='sdp-review__article__list__info__product-info__reg-date')

        try:
            # time.sleep(0.1)
            name = review.find_element(By.CSS_SELECTOR, '.sdp-review__article__list__info__user__name.js_reviewUserProfileImage')
            time.sleep(1)
        except:
            name = '없음'
        try:
            # time.sleep(0.1)
            member_id = review.find_element(By.CSS_SELECTOR, '.sdp-review__article__list__info__user__name.js_reviewUserProfileImage').get_attribute('data-member-id')
            # time.sleep(1)
            
        except:
            member_id = '없음'
        try:
            # time.sleep(0.1)
            star = review.find_element(By.CSS_SELECTOR, '.sdp-review__article__list__info__product-info__star-orange.js_reviewArticleRatingValue').get_attribute('data-rating')
            # time.sleep(1)
            
        except:
            star = '없음'
        try:
            # time.sleep(0.1)
            title = review.find_element(By.CSS_SELECTOR, '.sdp-review__article__list__headline')
            # time.sleep(1)
        except:
            title = '없음'
        try:
            # time.sleep(0.1)
            contents = review.find_element(By.CSS_SELECTOR, '.sdp-review__article__list__review__content.js_reviewArticleContent')
            # time.sleep(1)
        except:
            contents = '없음'
        try:
            # time.sleep(0.1)
            help = review.find_element(By.CSS_SELECTOR, '.js_reviewArticleHelpfulCount')
            # time.sleep(1)
        except:
            help = '없음'
        try:
            # time.sleep(0.1)
            date = review.find_element(By.CSS_SELECTOR, '.sdp-review__article__list__info__product-info__reg-date')
            # time.sleep(1)
        except:
            date = '없음'

        try:
            image = review.find_elements(By.CSS_SELECTOR, '.sdp-review__article__list__attachment__list')
            if len(image) == 0:
                image = 0
            else:
                image = 1
        except:
            image = 0

        name_2 = ''
        member_id_2 = ''
        star_2 = ''
        title_2 = ''
        contents_2 = ''
        help_2 = ''
        date_2 = ''
        if(name!='없음'):
            name_2=name.text.replace('\xa0', '')
        if(member_id!='없음'):
            member_id_2=member_id
        if(star!='없음'):
            star_2=star
        if(title!='없음'):
            title_2=title.text.strip().replace('\n', ' ')
        if(contents!='없음'):
            contents_2=contents.text.strip().replace('\n', ' ')
        if(help!='없음'):
            help_2=help.text
        if(date!='없음'):
            date_2=date.text
        # print([name_2, member_id_2, star_2, title_2, contents_2, help_2, date_2])
        
        product_reviews.append([name_2, member_id_2, star_2, title_2, contents_2, help_2, date_2, image])
    return product_reviews

    # except:
        # product_reviews = '해당 제품에 대한 리뷰가 없습니다.'
        # return product_reviews
        

In [15]:
def crawl_review_star(review_page_cnt):
    
    # 리뷰 페이지 한 페이지에 5개씩 있음 (페이지 넘기기 구현)
    # 페이지 번호에 따라 이전, 다음 버튼의 개수를 고려해야 함
    if review_page_cnt % 10 != 0:  
        button = review_page_cnt + (review_page_cnt // 10) * 2 + 1  # 페이지 번호가 27까지 있으면 for loop에서 range를 27 + 5 번 돌려야 함
    else:
        button = review_page_cnt + (review_page_cnt // 10) * 2 - 1 # 페이지 번호가 20까지 있으면 for loop에서 20 + 3 번 돌려야 함
    
    product_review_all = []
    # print(button)
    # print(review_page_cnt)
    if button == 2:
        time.sleep(1)
        reviews = driver.find_elements(By.CSS_SELECTOR, '.sdp-review__article__list.js_reviewArticleReviewList')
        time.sleep(4)
        product_review_all.append(review_data(reviews))
        time.sleep(4)
        
    else:
        for i in range(1, button + 1):
            # try:
                # driver.find_elements(By.CSS_SELECTOR, 'sdp-review__article__no-review.sdp-review__article__no-review--active')
            # except:
            if i % 12 == 1:  # 이전 버튼
                continue
            if i % 12 != 0 and i % 12 != 1:
                try:
                    time.sleep(1)
                    driver.find_element(By.XPATH, f'//*[@id="btfTab"]/ul[2]/li[2]/div/div[6]/section[4]/div[3]/button[{i % 12}]').click()
                    time.sleep(6)
                    reviews = driver.find_elements(By.CSS_SELECTOR, '.sdp-review__article__list.js_reviewArticleReviewList')
                    time.sleep(3)
                    product_review_all.append(review_data(reviews))
                except:
                    driver.find_elements(By.XPATH, f'//*[@id="btfTab"]/ul[2]/li[2]/div/div[6]/section[4]/div[3]/button[{i % 12}]')
                    time.sleep(5)
                    driver.find_element(By.XPATH, f'//*[@id="btfTab"]/ul[2]/li[2]/div/div[6]/section[4]/div[3]/button[{i % 12}]').click()
                    time.sleep(5)
                    reviews = driver.find_elements(By.CSS_SELECTOR, '.sdp-review__article__list.js_reviewArticleReviewList')
                    time.sleep(3)
                    product_review_all.append(review_data(reviews))
                    # time.sleep(3)
            if i % 12 == 0: # 다음 버튼
                try:
                    driver.find_elements(By.XPATH, f'//*[@id="btfTab"]/ul[2]/li[3]/div/div[6]/section[4]/div[3]/button[{12}]')[0].click()
                except:
                    driver.find_elements(By.XPATH, f'//*[@id="btfTab"]/ul[2]/li[3]/div/div[6]/section[4]/div[3]/button[{12}]')[0].click()
                    time.sleep(0.5)
    all_reviews = []
    for page in product_review_all:
        for review in page:
            all_reviews.append(review)
    # print(all_reviews)
    # driver.close()

    if all_reviews == []:
        return '해당 제품에 대한 리뷰가 없습니다.'
    else:
        return all_reviews

In [16]:
# url을 넣으면 좋음, 보통, 별로로 별점을 분류하여 리뷰들을 크롤링해 옴
# 리뷰 개수가 가장 적은 카테고리가 '별로'라고 했을 때, 리뷰의 개수가 15개보다 적으면 빈 리스트를 반환한다.
# 그리고 url list를 돌렸을 때, 빈 리스트를 가진 해당 데이터를 제외한다.
def crawl_star_three_cat(url):    
    global driver
    driver = webdriver.Chrome()
    driver.set_window_size(500, 500)
    # time.sleep(2)
    driver.get(url)
    time.sleep(2)
    
    min = min_star(url)

    # 가장 리뷰 개수가 적은 카테고리에서 리뷰 개수가 15개 이상인 것에서만 코드 실행행
    if min[1] >= 15:
        driver.find_element(By.XPATH, '//*[@id="btfTab"]/ul[1]/li[2]').click()  # 상품평 클릭
        time.sleep(2)
        
        driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star.js_reviewArticleSearchStarSelectBtn').click()
        star_click = []
        for i in range(1, 6):
            star_click.append(driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(i)))
        
        
        if min[0] == '좋음':
            star_click[0].click()
            review_page_cnt = int(np.ceil(min[1] / 5))
            reviews = crawl_review_star(review_page_cnt)
            
            positive_all = []
            for review in reviews:
                if (review[3] == '') and (review[4] == ''):
                    continue
                else:
                    positive_all.append(review)

            final_min = len(positive_all)

            if final_min >= 5:

                # final_min을 기준으로 좋음, 별로에 대한 리뷰들도 데이터를 가져온다.
                positive = []
                soso = []
                negative = []
                # positive reviews
                positive = positive_all
                # soso reviews
                try:
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(2)).click()
                except:
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star.js_reviewArticleSearchStarSelectBtn').click()
                    time.sleep(2)
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(2)).click()
                review_page_cnt = int(np.ceil(final_min / 5))
                reviews = crawl_review_star(review_page_cnt)
        
                for review in reviews:
                    if (review[3] == '') and (review[4] == ''):
                        continue
                    else:
                        soso.append(review)
                # negative reviews
                negative_all = []    
                for i in range(3, 6):
                    driver = webdriver.Chrome()
                    driver.set_window_size(500, 500)
                    # time.sleep(2)
                    driver.get(url)
                    time.sleep(2)
                    
                    driver.find_element(By.XPATH, '//*[@id="btfTab"]/ul[1]/li[2]').click()  # 상품평 클릭
                    time.sleep(5)
                    
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star.js_reviewArticleSearchStarSelectBtn').click()
                    time.sleep(3)
    
                    # 해당 별점 카테고리에 있는 리뷰의 개수를 가져온다.
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(i))
                    time.sleep(3)
                    star = driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(i))
                    time.sleep(3)
                    
                    count = star.text.replace(',', '')
                    counts = int(count)
                    
                    star.click()
                    negative_min_each = int(np.floor(final_min / 3))# 3개를 합쳐야됨
                    
                    # negative_min_each보다 리뷰의 개수가 적을 수 있기 때문에 경우의 수를 나눈다.
                    # 예를 들어, 별 3개, 2개, 1개 각각 10개의 리뷰를 가져와야 하는데 별 2개의 리뷰 개수가 9개인 경우 9개를 기준으로 해야 한다.
                    if negative_min_each >= counts:
                        review_page_cnt = int(np.ceil(counts / 5))
                        reviews = crawl_review_star(review_page_cnt)
                    else:
                        review_page_cnt = int(np.ceil(negative_min_each / 5))
                        reviews = crawl_review_star(review_page_cnt)
                    # 리뷰 제목과 리뷰 내용 둘 다 없는 리뷰는 제외하고 가져온다.
                    for review in reviews:
                        if (review[3] == '') and (review[4] == ''):
                            continue
                        else:
                            negative_all.append(review)
                negative = negative_all
                driver.close()
                return [positive, soso, negative]
            else:
                return 'None'
                
            
        
        elif min[0] == '보통':
            star_click[1].click()
            review_page_cnt = int(np.ceil(min[1] /5))
            reviews = crawl_review_star(review_page_cnt)

            soso_all = []
            for review in reviews:
                if (review[3] == '') and (review[4] == ''):
                    continue
                else:
                    soso_all.append(review)
            final_min = len(soso_all)
            # print(final_min)
            if final_min >= 5:

                # final_min을 기준으로 좋음, 별로에 대한 리뷰들도 데이터를 가져온다.
                positive = []
                soso = []
                negative = []
                # soso reviews
                soso = soso_all
                # positive reviews
                try:
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(1)).click()
                except:
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star.js_reviewArticleSearchStarSelectBtn').click()
                    time.sleep(2)
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(1)).click()
                review_page_cnt = int(np.ceil(final_min / 5))
                reviews = crawl_review_star(review_page_cnt)
        
                for review in reviews:
                    if (review[3] == '') and (review[4] == ''):
                        continue
                    else:
                        positive.append(review)
                # negative reviews
                negative_all = []    
                for i in range(3, 6):
                    driver = webdriver.Chrome()
                    driver.set_window_size(500, 500)
                    # time.sleep(2)
                    driver.get(url)
                    time.sleep(2)
                    
                    driver.find_element(By.XPATH, '//*[@id="btfTab"]/ul[1]/li[2]').click()  # 상품평 클릭
                    time.sleep(5)
                    
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star.js_reviewArticleSearchStarSelectBtn').click()
                    time.sleep(3)
    
                    # 해당 별점 카테고리에 있는 리뷰의 개수를 가져온다.
                    star = driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(i))
                    time.sleep(3)
                    
                    count = star.text.replace(',', '')
                    counts = int(count)
                    # print(counts)
                    star.click()
                    negative_min_each = int(np.floor(final_min / 3))# 3개를 합쳐야됨
                    # print(negative_min_each)
                    # negative_min_each보다 리뷰의 개수가 적을 수 있기 때문에 경우의 수를 나눈다.
                    # 예를 들어, 별 3개, 2개, 1개 각각 10개의 리뷰를 가져와야 하는데 별 2개의 리뷰 개수가 9개인 경우 9개를 기준으로 해야 한다.
                    if negative_min_each >= counts:
                        review_page_cnt = int(np.ceil(counts / 5))
                        reviews = crawl_review_star(review_page_cnt)
                    else:
                        review_page_cnt = int(np.ceil(negative_min_each / 5))
                        reviews = crawl_review_star(review_page_cnt)
                    # 리뷰 제목과 리뷰 내용 둘 다 없는 리뷰는 제외하고 가져온다.
                    for review in reviews:
                        if (review[3] == '') and (review[4] == ''):
                            continue
                        else:
                            negative_all.append(review)
                negative = negative_all
                driver.close()
                return [positive, soso, negative]
            else:
                return 'None'

        
        elif min[0] == '별로':
            negative_all = []    
            for i in range(3, 6):
                driver = webdriver.Chrome()
                driver.set_window_size(500, 500)
                # time.sleep(2)
                driver.get(url)
                time.sleep(2)
                
                driver.find_element(By.XPATH, '//*[@id="btfTab"]/ul[1]/li[2]').click()  # 상품평 클릭
                time.sleep(5)
                
                driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star.js_reviewArticleSearchStarSelectBtn').click()
                time.sleep(3)

                # 해당 별점 카테고리에 있는 리뷰의 개수를 가져온다.
                
                star = driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(i))
                time.sleep(3)

                
                count = star.text.replace(',', '')
                counts = int(count)
                
                star.click()
                negative_min_each = int(np.floor(min[1] / 3))# 3개를 합쳐야됨
                
                # negative_min_each보다 리뷰의 개수가 적을 수 있기 때문에 경우의 수를 나눈다.
                # 예를 들어, 별 3개, 2개, 1개 각각 10개의 리뷰를 가져와야 하는데 별 2개의 리뷰 개수가 9개인 경우 9개를 기준으로 해야 한다.
                if negative_min_each >= counts:
                    review_page_cnt = int(np.ceil(counts / 5))
                    reviews = crawl_review_star(review_page_cnt)
                else:
                    review_page_cnt = int(np.ceil(negative_min_each / 5))
                    reviews = crawl_review_star(review_page_cnt)

                # 리뷰 제목과 리뷰 내용 둘 다 없는 리뷰는 제외하고 가져온다.
                for review in reviews:
                    if (review[3] == '') and (review[4] == ''):
                        continue
                    else:
                        negative_all.append(review)
                        
            # 별점만 있는 리뷰를 제외한 부정 리뷰 데이터 최종 개수
            final_min = len(negative_all)

            if final_min > 5:
            
                # final_min을 기준으로 좋음, 보통에 대한 리뷰들도 데이터를 가져온다.
                positive = []
                soso = []
                negative = []
                # negative reviews
                negative = negative_all
                # positive 
                try:
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(1)).click()
                except:
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star.js_reviewArticleSearchStarSelectBtn').click()
                    time.sleep(2)
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(1)).click()
                review_page_cnt = int(np.ceil(final_min / 5))
                reviews = crawl_review_star(review_page_cnt)
            
                for review in reviews:
                    if (review[3] == '') and (review[4] == ''):
                        continue
                    else:
                        positive.append(review)
                # soso reviews
                try:
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(2)).click()
                except:
                    driver.find_element(By.XPATH, '//*[@id="btfTab"]/ul[1]/li[2]').click()  # 상품평 클릭
                    time.sleep(2)
                    
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star.js_reviewArticleSearchStarSelectBtn').click()
                    time.sleep(2)
                    driver.find_element(By.CSS_SELECTOR, '.sdp-review__article__order__star__list__item__count.js_reviewArticleOptionStarCount' + str(2)).click()
                review_page_cnt = int(np.ceil(final_min / 5))
                reviews = crawl_review_star(review_page_cnt)
        
                for review in reviews:
                    if (review[3] == '') and (review[4] == ''):
                        continue
                    else:
                        soso.append(review)
                # print(len(positive))
                # print(len(soso))
                # print(len(negative))
                driver.close()
                return [positive, soso, negative]
            else:
                return 'None'
    else:
        return 'None'  # 리뷰 개수가 적을 때 None 반환

In [17]:
def review(csv):   # csv에는 카테고리별로 생성된 
    review = csv
    review = review[review['리뷰 개수'] >= 45].reset_index()
    
    all = pd.DataFrame()
    categories = review['카테고리2'].drop_duplicates().reset_index()
    for i in range(len(categories)):
        category = review[review['카테고리2'] == categories.loc[i][1]][:15]
        all = pd.concat([all, category])
    all = all.reset_index()
    review = all
    
    review_list = []
    p_review_counts = []
    s_review_counts = []
    n_review_counts = []
    for i in range(len(review)):
        review_list.append([])
        p_review_counts.append([])
        s_review_counts.append([])
        n_review_counts.append([])
    i = 0
    for url in fashion_products_review.loc[:]['제품 url']:
        print(i, url)
        crawl = crawl_star_three_cat(url)
        review_list[i] = crawl
        if crawl == 'None':
            i += 1
            continue
        else:
            positive = len(crawl[0])
            soso = len(crawl[1])
            negative = len(crawl[2])
            p_review_counts[i] = positive
            s_review_counts[i] = soso
            n_review_counts[i] = negative
        i += 1
    review['리뷰'] = review_list
    review['리뷰(좋음)'] = p_review_counts
    review['리뷰(보통)'] = s_review_counts
    review['리뷰(별로)'] = n_review_counts
    
    drop_duplicates = review.drop_duplicates(['리뷰'], keep='first')
    total_review = drop_duplicates[drop_duplicates['리뷰'] != 'None']

    total_review = total_review[total_review['리뷰'] != '[]']
    total_review = total_review[total_review['리뷰(좋음)'].notnull()]
    return total_review

In [18]:
# file_dir='/coupang_data/products/'
# file_name='products_list_'+cal_std_day(1)+'.csv'
# products = spark.read.csv(file_dir+file_name)
# products.show()

## 1. 원본 파일

In [19]:
# 완전 원본 파일(상품 + 리뷰 데이터)
raw_data = spark.read.option("header", "true").csv('/final/total_data_ver2.csv')
raw_data.show()

+---+-----------+------------------+--------------------+-----------------------------------+--------------------+----------+--------------+-------------------------+--------------------------------+---------------------------------+---------------------------------+
|_c0|  category1|         category2|        category_url|                       product_name|         product_url|review_cnt|product_rating|                   review|                     review_good|                      review_soso|                       review_bad|
+---+-----------+------------------+--------------------+-----------------------------------+--------------------+----------+--------------+-------------------------+--------------------------------+---------------------------------+---------------------------------+
|  0|출산/유아동|         수유용품 |http://www.coupan...|      그로미미 PPSU 젖병 트윈팩,...|http://www.coupan...|      1978|           5.0|"[[['리뷰호랭이 ', '65...| 유니콘처럼 완벽하진 않아요."...|  그게 좀 아쉽다면 다른 옵션도...|  준비된 엄마라면

In [20]:
raw_data.columns

['_c0',
 'category1',
 'category2',
 'category_url',
 'product_name',
 'product_url',
 'review_cnt',
 'product_rating',
 'review',
 'review_good',
 'review_soso',
 'review_bad']

In [21]:
raw_data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- category1: string (nullable = true)
 |-- category2: string (nullable = true)
 |-- category_url: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_url: string (nullable = true)
 |-- review_cnt: string (nullable = true)
 |-- product_rating: string (nullable = true)
 |-- review: string (nullable = true)
 |-- review_good: string (nullable = true)
 |-- review_soso: string (nullable = true)
 |-- review_bad: string (nullable = true)



## 2. 전체 상품 데이터

In [54]:
products = spark.read.option("header", "true").csv('/final/products_total.csv')
products.show()

+-----------+-----------+--------------------+----------------------------------+--------------------+------------+--------------+
|  category1|  category2|        category_url|                      product_name|         product_url|review_count|product_rating|
+-----------+-----------+--------------------+----------------------------------+--------------------+------------+--------------+
|출산/유아동|유아동패션 |http://www.coupan...|     꼰띠키즈 여아용 다이어리 데님|http://www.coupan...|          42|           4.5|
|출산/유아동|유아동패션 |http://www.coupan...| 시나모롤 큐티 소풍가방 어린이 ...|http://www.coupan...|         121|           5.0|
|출산/유아동|유아동패션 |http://www.coupan...|    휠라 아동용 꾸미 라이트 PT ...|http://www.coupan...|        2089|           5.0|
|출산/유아동|유아동패션 |http://www.coupan...|   휠라 아동용 커블렛 운동화 2....|http://www.coupan...|         109|           5.0|
|출산/유아동|유아동패션 |http://www.coupan...|     이설한복 50001 그린지호 남...|http://www.coupan...|           3|           5.0|
|출산/유아동|유아동패션 |http://www.coupan...|          초코몽 아동용 메이슨데

In [23]:
products.columns

['category1',
 'category2',
 'category_url',
 'product_name',
 'product_url',
 'review_count',
 'product_rating']

## 3. 리뷰 데이터 펼친 csv 파일에 test용 10명 리뷰 데이터 추가(멤버 id로 구분)

In [63]:
customers = spark.read.option("delimiter", "+").option("lineSep", "@").option("header", "true").option("quote", "\"").csv('/final/reviews_total.csv')
customers.show()

+-------------+------------------+----------------------------------+-----------+----------+-------------+------+-------+----+----+--------------------------------+
|    category1|         category2|                      product_name|product_url|rating_cnt|reviewer_name|userid|ratings|help|date|                 review_contents|
+-------------+------------------+----------------------------------+-----------+----------+-------------+------+-------+----+----+--------------------------------+
|헬스/건강식품|  곤약/방탄커피 외|          국내산 시금치, 200g, 1개|       None|      None|         None| cust1|      3|None|None|  아삭해요~◈ 제품명 : 국내산 ...|
|     결혼준비|          주방용품|  리한 오뚝이 롤리 주걱, 화이트...|       None|      None|         None| cust1|      3|None|None|위생적이고 엄청 편해요~◈ 제품...|
|헬스/건강식품|  곤약/방탄커피 외|              밀양 깻잎, 100g, 1개|       None|      None|         None| cust1|      3|None|None|  향이 좋아요~◈ 제품명 : 밀양...|
|         식품|              채소|         국내산 양배추, 1개입, 1개|       None|      None|         None| cust1|

In [25]:
!pip install pymysql

Defaulting to user installation because normal site-packages is not writeable


In [1]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 
import pymysql

/home/lab04/.local/lib/python3.6/site-packages/pymysql/_auth.py:8: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.hazmat.backends import default_backend


In [2]:
JDBC = {
      'url':'jdbc:mysql://localhost:3306/final_project?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigMysql',
      'password':'bigMysql1234@'   
      }
}
conf_dm = {
      'url':'jdbc:mysql://localhost:3306/final_project_DM?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigDM',
      'password':'bigDM1234@'   
      }
}

In [9]:
raw_data = spark.read.option("header", "true").csv('/final/total_data_ver2.csv')
products = spark.read.option("header", "true").csv('/final/products_total.csv')
customers = spark.read.option("delimiter", "+").option("lineSep", "@").option("header", "true").option("quote", "\"").csv('/final/reviews_total.csv')

In [11]:
products.createOrReplaceTempView('products')
customers.createOrReplaceTempView('customers')

In [13]:
products.show()

+-----------+-----------+--------------------+----------------------------------+--------------------+------------+--------------+
|  category1|  category2|        category_url|                      product_name|         product_url|review_count|product_rating|
+-----------+-----------+--------------------+----------------------------------+--------------------+------------+--------------+
|출산/유아동|유아동패션 |http://www.coupan...|     꼰띠키즈 여아용 다이어리 데님|http://www.coupan...|          42|           4.5|
|출산/유아동|유아동패션 |http://www.coupan...| 시나모롤 큐티 소풍가방 어린이 ...|http://www.coupan...|         121|           5.0|
|출산/유아동|유아동패션 |http://www.coupan...|    휠라 아동용 꾸미 라이트 PT ...|http://www.coupan...|        2089|           5.0|
|출산/유아동|유아동패션 |http://www.coupan...|   휠라 아동용 커블렛 운동화 2....|http://www.coupan...|         109|           5.0|
|출산/유아동|유아동패션 |http://www.coupan...|     이설한복 50001 그린지호 남...|http://www.coupan...|           3|           5.0|
|출산/유아동|유아동패션 |http://www.coupan...|          초코몽 아동용 메이슨데

In [6]:
from pyspark.sql.functions import monotonically_increasing_id
products = products.withColumn('product_id', monotonically_increasing_id())

In [7]:
from pyspark.sql.functions import monotonically_increasing_id
customers = customers.withColumn('review_id', monotonically_increasing_id())

In [8]:
products.createOrReplaceTempView('products')
customers.createOrReplaceTempView('customers')

In [14]:
products.write.mode('overwrite').jdbc(url=JDBC['url'], table='products', properties=JDBC['props'])

In [15]:
customers.write.mode('overwrite').jdbc(url=JDBC['url'], table='customers', properties=JDBC['props'])

In [69]:
products.printSchema()

root
 |-- category1: string (nullable = true)
 |-- category2: string (nullable = true)
 |-- category_url: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_url: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- product_rating: string (nullable = true)
 |-- product_id: long (nullable = false)



In [6]:
products_table = spark.sql('''
    select product_id, product_name, product_url, category1, category2, category_url
    from products
    WHERE category1 IN (
        SELECT category1
        FROM (
            SELECT category1, COUNT(category1) as category_count
            FROM products
            GROUP BY category1
            HAVING category_count > 1
        )
    );
''')

In [7]:
products_table.createOrReplaceTempView('products_table')

In [8]:
spark.sql('''
    SELECT category1, count(*) FROM products_table
    GROUP BY category1;
''').show()

+-------------+--------+
|    category1|count(1)|
+-------------+--------+
|    완구/취미|    8095|
|   자동차용품|    5848|
|헬스/건강식품|    6783|
|  출산/유아동|    9491|
|패션의류/잡화|    1780|
|도서/음반/DVD|    9206|
|         뷰티|    8468|
|   가전디지털|    8824|
|         식품|    9356|
|  스포츠/레저|    7415|
|  문구/오피스|    7343|
|   홈인테리어|    7656|
|     생활용품|    5726|
| 반려동물용품|    6271|
|     주방용품|    5963|
+-------------+--------+



In [73]:
spark.sql('''
    SELECT count(*) FROM products_table;
''').show()

+--------+
|count(1)|
+--------+
|  108225|
+--------+



In [9]:
products_table.write.mode('overwrite').jdbc(url=conf_dm['url'], table='products_table', properties=conf_dm['props'])

In [11]:
products_review_info = spark.sql('''
    SELECT tb1.product_id, tb1.product_rating, tb1.review_count
    FROM products tb1
    JOIN products_table tb2 ON tb1.product_id == tb2.product_id;
''')

In [12]:
products_review_info.createOrReplaceTempView('products_review_info')

In [13]:
spark.sql('''
    select count(*) from products_review_info;
''').show()

+--------+
|count(1)|
+--------+
|  108225|
+--------+



In [14]:
products_review_info.write.mode('overwrite').jdbc(url=conf_dm['url'], table='products_review_info', properties=conf_dm['props'])

In [79]:
products_table.printSchema()

root
 |-- product_id: long (nullable = false)
 |-- product_name: string (nullable = true)
 |-- product_url: string (nullable = true)
 |-- category1: string (nullable = true)
 |-- category2: string (nullable = true)
 |-- category_url: string (nullable = true)



In [80]:
customers.printSchema()

root
 |-- category1+category2+product_name+product_url+rating_cnt+reviewer_name+userid+ratings+help+date+review_contents@@헬스/건강식품+곤약/방탄커피 외+국내산 시금치: string (nullable = true)
 |--  200g1: string (nullable = true)
 |--  1개+None+None+None+cust1+3+None+None+아삭해요~◈ 제품명 : 국내산 시금치: string (nullable = true)
 |--  200g3: string (nullable = true)
 |--  1개: string (nullable = true)
 |-- review_id: long (nullable = false)



In [15]:
reviews_table = spark.sql('''
    SELECT tb1.review_id, tb1.reviewer_name, tb1.userid, tb1.review_contents, tb1.ratings, tb1.product_name, tb1.category1, tb1.category2, tb2.product_id
    FROM customers tb1
    LEFT JOIN products_table tb2 ON tb1.product_url == tb2.product_url
    WHERE tb1.category1 in (SELECT category1 FROM products_table
    GROUP BY category1);
''')

In [16]:
reviews_table.createOrReplaceTempView('reviews_table')

In [17]:
spark.sql('''
    select count(*) from reviews_table;
''').show()

+--------+
|count(1)|
+--------+
|  159244|
+--------+



In [108]:
customers.show()

+-------------+------------------+----------------------------------+-----------+----------+-------------+------+-------+----+----+--------------------------------+---------+
|    category1|         category2|                      product_name|product_url|rating_cnt|reviewer_name|userid|ratings|help|date|                 review_contents|review_id|
+-------------+------------------+----------------------------------+-----------+----------+-------------+------+-------+----+----+--------------------------------+---------+
|헬스/건강식품|  곤약/방탄커피 외|          국내산 시금치, 200g, 1개|       None|      None|         None| cust1|      3|None|None|  아삭해요~◈ 제품명 : 국내산 ...|        0|
|     결혼준비|          주방용품|  리한 오뚝이 롤리 주걱, 화이트...|       None|      None|         None| cust1|      3|None|None|위생적이고 엄청 편해요~◈ 제품...|        1|
|헬스/건강식품|  곤약/방탄커피 외|              밀양 깻잎, 100g, 1개|       None|      None|         None| cust1|      3|None|None|  향이 좋아요~◈ 제품명 : 밀양...|        2|
|         식품|              채소|         국

In [109]:
reviews_table.show(100)

+---------+-------------+------+----------------------------------+-------+-----------------------------------+-------------+-------------------+----------+
|review_id|reviewer_name|userid|                   review_contents|ratings|                       product_name|    category1|          category2|product_id|
+---------+-------------+------+----------------------------------+-------+-----------------------------------+-------------+-------------------+----------+
|        0|         None| cust1|    아삭해요~◈ 제품명 : 국내산 ...|      3|           국내산 시금치, 200g, 1개|헬스/건강식품|   곤약/방탄커피 외|      null|
|        2|         None| cust1|    향이 좋아요~◈ 제품명 : 밀양...|      3|               밀양 깻잎, 100g, 1개|헬스/건강식품|   곤약/방탄커피 외|      null|
|        3|         None| cust1|    달콤해요~◈ 제품명 : 국내산 ...|      3|          국내산 양배추, 1개입, 1개|         식품|               채소|      null|
|        4|         None| cust1|   파릇파릇해요~◈ 제품명 : 국내...|      3|             국내산 부추, 300g, 1개|헬스/건강식품|   곤약/방탄커피 외|      null|
|        5| 

In [18]:
reviews_table.write.mode('overwrite').jdbc(url=conf_dm['url'], table='reviews_table', properties=conf_dm['props'])

### products 데이터 프레임 => 3개의 table 생성(FK로 연결)
CREATE TABLE first_category AS
SELECT DISTINCT category1
FROM products;
ALTER TABLE first_category ADD PRIMARY KEY(category1);

CREATE TABLE second_category (
    category1 VARCHAR(50),
    category2 VARCHAR(50),
    category_url VARCHAR(500),
    PRIMARY KEY (category2, category_url)
) AS
SELECT DISTINCT category1, category2, category_url
FROM products;
ALTER TABLE second_category ADD CONSTRAINT fk_category1 FOREIGN KEY (category1) REFERENCES first_category(category1);

CREATE TABLE products_data (
    product_name VARCHAR(500),
    product_url VARCHAR(500),
    review_count INT,
    product_rating FLOAT,
    category2 VARCHAR(50),
    PRIMARY KEY (product_name, product_url)
) AS
SELECT product_name, product_url, review_count, product_rating, category2
FROM products;
ALTER TABLE products_data ADD CONSTRAINT fk_category2 FOREIGN KEY (category2) REFERENCES second_category(category2);

In [1]:
# products.createOrReplaceTempView('products')

In [5]:
# products.write.mode('overwrite').jdbc(url=JDBC['url'], table='products', properties=JDBC['props'])

In [2]:
# # category1 정보만 있는 table
# table_category1 = spark.sql('''
#     SELECT category1
#     FROM products
#     GROUP BY category1
#     HAVING count(category1) > 1;
# ''')

In [3]:
# # category2, category_url, category1(FK)로 가지고 있는 table
# cat2 = spark.sql('''
#     select DISTINCT category2, category_url, category1
#     from products;
# ''')

In [4]:
# # category1으로 join
# table_category1.createOrReplaceTempView("category1_view")
# cat2.createOrReplaceTempView("category2_view")

# table_category2 = spark.sql('''
#     SELECT DISTINCT tc2.category2, tc2.category_url, tc1.category1
#     FROM category2_view tc2
#     JOIN category1_view tc1 ON tc2.category1 = tc1.category1
# ''')

In [5]:
# # product_name, product_url, review_count, product_rating, category2 있는 table
# products_info = spark.sql('''
#     SELECT DISTINCT product_name, product_url, review_count, product_rating, category2
#     FROM products;
# ''')

In [6]:
# table_category2.createOrReplaceTempView("table_category2")
# products_info.createOrReplaceTempView("products_info")

# # category2로 join
# table_category3 = spark.sql('''
#     SELECT DISTINCT tc1.product_name, tc1.product_url, tc1.review_count, tc1.product_rating, tc2.category2
#     FROM products_info tc1
#     JOIN table_category2 tc2 ON tc1.category2 = tc2.category2;
# ''')

In [7]:
# table_category1.write.mode('overwrite').jdbc(url=JDBC['url'], table='products_table_1', properties=JDBC['props'])

In [8]:
# table_category2.write.mode('overwrite').jdbc(url=JDBC['url'], table='products_table_2', properties=JDBC['props'])

In [9]:
# table_category3.write.mode('overwrite').jdbc(url=JDBC['url'], table='products_table_3', properties=JDBC['props'])

## reviews 파일 이용

In [41]:
# customers.write.mode('overwrite').jdbc(url=JDBC['url'], table='customers', properties=JDBC['props'])

In [10]:
# customers.createOrReplaceTempView('customers')

In [11]:
# customers.printSchema()

In [12]:
# customers.show()

In [13]:
# # category1 정보만 있는 table
# reviews_table_category1 = spark.sql('''
#     SELECT DISTINCT category1
#     FROM customers
#     WHERE category1 IN ('출산/유아동', '뷰티', '도서/음반/DVD', '자동차용품', '가전디지털', '패션의류/잡화', '식품','완구/취미', '헬스/건강식품', '홈인테리어', '주방용품', '생활용품', '문구/오피스', '반려동물용품','스포츠/레저')
#     GROUP BY category1;
# ''')

In [14]:
# reviews_table_category1.createOrReplaceTempView('reviews_table_category1')

In [15]:
# reviews_cat2 = spark.sql('''
#     select DISTINCT category2, category1
#     from customers;
# ''')

In [16]:
# # category1으로 join
# reviews_table_category1.createOrReplaceTempView("category1_view")
# reviews_cat2.createOrReplaceTempView("category2_view")

# reviews_table_category2 = spark.sql('''
#     SELECT DISTINCT tc2.category2, tc1.category1
#     FROM category2_view tc2
#     JOIN category1_view tc1 ON tc2.category1 = tc1.category1
# ''')

In [17]:
# reviews_table_category2.createOrReplaceTempView('reviews_table_category2')

In [18]:
# customers.printSchema()

In [19]:

# reviews_info = spark.sql('''
#     SELECT DISTINCT product_name, review_cnt, reviewer_name, userid, ratings, review_contents, category2
#     FROM customers;
# ''')

In [20]:
# reviews_table_category2.createOrReplaceTempView("reviews_table_category2")
# reviews_info.createOrReplaceTempView("reviews_info")

# # category2로 join
# reviews_table_category3 = spark.sql('''
#     SELECT DISTINCT tc1.product_name, tc1.review_cnt, tc1.reviewer_name, tc1.userid, tc1.ratings, tc1.review_contents, tc2.category2
#     FROM reviews_info tc1
#     JOIN reviews_table_category2 tc2 ON tc1.category2 = tc2.category2;
# ''')

In [21]:
# reviews_table_category1.write.mode('overwrite').jdbc(url=JDBC['url'], table='reviews_table_category1', properties=JDBC['props'])

In [22]:
# reviews_table_category2.write.mode('overwrite').jdbc(url=JDBC['url'], table='reviews_table_category2', properties=JDBC['props'])

In [23]:
# reviews_table_category3.write.mode('overwrite').jdbc(url=JDBC['url'], table='reviews_table_category3', properties=JDBC['props'])

In [ ]:
# # dm db에 저장할 predict 테이블
# predict = customers.select(col('review_contents').alias('REVIEW')
#                           , col('ratings').alias('RATING'))

In [ ]:
# predict.write.mode('overwrite').jdbc(url=JDBC['url'], table='predict', properties=JDBC['props'])
# predict.show()